In [1]:
import math
import collections
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

%matplotlib inline

In [11]:

from google.colab import drive#to open the google drive in colab
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
lines=[]
with open('/content/drive/My Drive/Case/nanoBloodProject/brushBox/NP/NPbonds/new_for_ML_project/np_bonds/rod.dat') as fp:   # to read lines if atomlist after "Atoms"
         for line in fp:
            if 'Atoms' in line:                                         
                for line in fp: # now you are at the lines you want
                    if 'Velocities' in line: 
                     break 
                   
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    #
                    lines.append(line)# add the string in to the list
                    #lines= line.split("space")# for line in a pandas data frame. here it can not read our lines separetely. it reads all lines as a line so does not split it into different indexs

In [18]:
#remove empty lines from the list
while("" in lines) : 
    lines.remove("") 

In [19]:
df=pd.DataFrame(lines, columns=['atomID']) # to make a data frame of pandas from the list and put atomID name on the column
df[['atomID','molID','x','y','z','mx','my','mz']] = df.atomID.str.split(expand=True,) # to split one column to multiple columns and allocate name for each column
df

,atomID,molID,x,y,z,mx,my,mz
0,1,1,-1.4958340847941662e+00,0.0000000000000000e+00,-5.9833363391766650e+00,0,0,0
1,2,1,-1.1218755635956246e+00,-6.4771515580875083e-01,-5.9833363391766650e+00,0,0,0
2,3,1,-7.4791704239708312e-01,-1.2954303116175017e+00,-5.9833363391766650e+00,0,0,0
3,4,1,-1.1218755635956246e+00,6.4771515580875083e-01,-5.9833363391766650e+00,0,0,0
4,5,1,-7.4791704239708312e-01,0.0000000000000000e+00,-5.9833363391766650e+00,0,0,0
...,...,...,...,...,...,...,...,...
666,667,1,7.4791704239708312e-01,0.0000000000000000e+00,5.9833363391766650e+00,0,0,0
667,668,1,1.1218755635956246e+00,-6.4771515580875083e-01,5.9833363391766650e+00,0,0,0
668,669,1,7.4791704239708312e-01,1.2954303116175017e+00,5.9833363391766650e+00,0,0,0
669,670,1,1.1218755635956246e+00,6.4771515580875083e-01,5.9833363391766650e+00,0,0,0


In [20]:
atom_counter=len(df)
atom_counter

671

In [21]:
Xmsd=np.zeros ((atom_counter,atom_counter))# make 2D arrays
Ymsd=np.zeros ((atom_counter,atom_counter))
Zmsd=np.zeros ((atom_counter,atom_counter))
Rmsd=np.zeros ((atom_counter,atom_counter))
# #double array bc we want compare distances for each i and j  with each other
min_Rmsd=np.zeros (atom_counter)
min_all=np.zeros (atom_counter)

df.loc[:,'x'] = df.loc[:,'x'].astype(float) #convert string to foat for x , y , z 
df.loc[:,'y'] = df.loc[:,'y'].astype(float)
df.loc[:,'z'] = df.loc[:,'z'].astype(float)

#smallest = float("inf") # infinity


for i in range(0, atom_counter-1):
  smallest = float("inf") # infinity. to renew after each i to just find min of each i not min of the whole.
  for j in range(i+1, atom_counter):
  
    dx=df.loc[j,'x']-df.loc[i,'x']
    dy=df.loc[j,'y']-df.loc[i,'y']
    dz=df.loc[j,'z']-df.loc[i,'z']

    Xmsd[i,j]+=np.power(dx,2)
    Ymsd[i,j]+=np.power(dy,2)
    Zmsd[i,j]+=np.power(dz,2)
        
    #distance of all beads from each other without repeating 2-1 after calc distance of 1-2. it is distance of 3 and 4 and is not distance 3 ordistance of 4. Rmsd[3,4] and not Rmsd[3] or Rmsd[4]
    Rmsd[i,j]=math.sqrt(Xmsd[i,j]+Ymsd[i,j]+ Zmsd[i,j]);
    
    #find the min of Rmsd array
    num= float(Rmsd[i,j])    
    smallest = min(smallest,num)
    min_Rmsd[i]=smallest
    #print (j,"  ",i,"  ",Rmsd[i,j],"   ",min_Rmsd[i])
      

   

In [77]:
#to find min of all min_Rmsd
smallest = float("inf") # infinity
for i in range(0, atom_counter-1): # we have five bond (distance) between 6 beads
    num= float(min_Rmsd[i])    
    smallest = min(smallest,num)
    min_all[i]=smallest
    smallest_bead_distance= round(min_all[i],4)# round the float to 4 decimal
print (i,"    ",smallest_bead_distance)

5375      0.7479


In [52]:
#here we find beads that have the min of distances and then count number of bonds with that size( distance) each bead has 
#count_bonded=np.zeros(atom_counter)

#for i in range(0, atom_counter-1):  
  #for j in range(i+1, atom_counter):
    #if round(Rmsd[i,j],4) == smallest_bead_distance:
    #if round(Rmsd[i,j],4) == 0.7251:
      #count_bonded[i]+=1
     
     # print(i,"   ",j,"   ",df.loc[i,'atomID'],"  ",count_bonded[i])

In [ ]:
# show the beads with the least distance with each bead in front of that bead

#bonded_beads=np.zeros((atom_counter,atom_counter))
#bonded_beads_4=np.zeros((atom_counter,atom_counter))

#for i in range(0, atom_counter-1): 
  #for j in range(i+1, atom_counter):
    #for k in range(0, 5):

      #if round(Rmsd[i,j],4) == smallest_bead_distance:
        
           #bonded_beads [i,k]= df.loc[j,'atomID']  # to save the beads that have least distance with a bead in an array for that bead
        
      
         
      
           #print(i,"   ",j,"   ",k,"   ",bonded_beads [i,k])
      

In [ ]:
# show the beads with the least distance with each bead in front of that bead and print in a file
#counter_bond=0 # to count total number of  bonds
#bonded_beads=np.zeros((atom_counter,atom_counter))
#bonded_beads_4=np.zeros((atom_counter,atom_counter))
#with open('/content/drive/My Drive/Case/nanoBloodProject/brushBox/NP/hexagonal_np/one_atom_type/simple_lattice_structure/very_simple_lattice/5937beads/bonds.dat', 'w') as f:

  #for i in range(0, atom_counter-1): 
    #counter=0
    
    #for j in range(i+1, atom_counter):
      
       
      #if round(Rmsd[i,j],4) == smallest_bead_distance:
           #counter+=1   # thank GOD instead of making another for loop to go through the 4 bonded beads we used this counter annd it will only save bonded beads in the array
           #counter_bond+=1

           #bonded_beads [i][j]= df.loc[j,'atomID']  # to save the beads that have least distance with a bead in an array for that bead
           #bonded_beads_4[i][counter]=bonded_beads[i][j]
           
           #bonded_beads_4 [i][j]= bonded_beads [i]
           
      
          
           #print( counter_bond,"   ", 1,"   ",df.loc[i,'atomID'],"    ",int(bonded_beads_4[i][counter]))
           
# to print in a file
           #original_stdout = sys.stdout # Save a reference to the original standard output
           #sys.stdout = f # Change the standard output to the file we created.
           #print( counter_bond,"   ", 1,"   ",df.loc[i,'atomID'],"    ",int(bonded_beads_4[i][counter]))
           #ys.stdout = original_stdout # Reset the standard output to its original value


In [78]:
#here we combine two tasks above to make the code faster and so we do:
#1) find beads that have the min of distances and then count number of bonds with that size( distance) each bead has 
#2)show the beads with the least distance with each bead in front of that bead and print in a file
counter_bond=0 # to count total number of  bonds
bonded_beads=np.zeros((atom_counter,atom_counter))
bonded_beads_4=np.zeros((atom_counter,atom_counter))
count_bonded=np.zeros(atom_counter)
with open('/content/drive/My Drive/Case/nanoBloodProject/brushBox/NP/hexagonal_np/one_atom_type/simple_lattice_structure/very_simple_lattice/new_for_ML_project/np_bonds/bonds.dat', 'w') as f:



 for i in range(0, atom_counter-1):
   counter=0  
   for j in range(i+1, atom_counter):

    if round(Rmsd[i,j],4) == smallest_bead_distance:
       counter+=1   # thank GOD instead of making another for loop to go through the 4 bonded beads we used this counter annd it will only save bonded beads in the array
       counter_bond+=1
       count_bonded[i]+=1

       bonded_beads [i][j]= df.loc[j,'atomID']  # to save the beads that have least distance with a bead in an array for that bead
       bonded_beads_4[i][counter]=bonded_beads[i][j]
       #print(i,"   ",j,"   ",df.loc[i,'atomID'],"  ",count_bonded[i])
       # to print in a file
       original_stdout = sys.stdout # Save a reference to the original standard output
       sys.stdout = f # Change the standard output to the file we created.
       print( counter_bond,"   ", 1,"   ",df.loc[i,'atomID'],"    ",int(bonded_beads_4[i][counter]))
       sys.stdout = original_stdout # Reset the standard output to its original value